In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [2]:
chat.predict("How many planets are in the solar system?")

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert, and you can speak only in Korean. Your name is 이도."
    ),
    AIMessage(content="안녕하세요, 지리에 관해서는 무엇이든 물어보세요!"),
    HumanMessage(
        content="우리나라에서 가장 멀리 떨어져 있는 나라는 어디인가요? 그리고 지리의 달인이시라고 들었는데, 성함이 어떻게 되시죠?"
    ),
]

chat.predict_messages(messages)

AIMessage(content='우리나라에서 가장 멀리 떨어져 있는 나라는 아르헨티나입니다. 저의 이름은 이도입니다. 지리에 관련된 질문이 있으면 언제든지 물어보세요!')

In [11]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a geography expert, and you speak only in {language}. Your name is {name}.",
        ),
        ("ai", "안녕하세요, 지리에 관해서는 무엇이든 물어보세요!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}? Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Spanish", name="Fernando", country_a="Columbia", country_b="Italy"
)

chat.predict_messages(prompt)

AIMessage(content='¡Hola! Mi nombre es Fernando. La distancia entre Colombia e Italia es de aproximadamente 9,400 kilómetros. ¿Hay algo más en lo que pueda ayudarte?')

In [15]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generator. Your answer must be a comma-separated list of maximum {max_items} items in lowercase. Do NOT answer with any other format.",
        ),
        ("human", "{question}"),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke(
    {
        "max_items": 5,
        "question": "What are the names of the drivers in Formula 1?",
    }
)

['lewis hamilton',
 'max verstappen',
 'valtteri bottas',
 'sergio perez',
 'charles leclerc']

In [17]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy-to-follow recipes for any type of cuisine with easy-to-find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specializing in making original recipes vegetarian. You find alternative ingredients and explain how to prepare them. You don't radically modify the recipe. If there is no suitable alternative, just say you can't help with that.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain

final_chain.invoke({"cuisine": "Indian"})

AIMessage(content="For a vegetarian alternative to the boneless chicken in the Butter Chicken recipe, you can substitute it with paneer, a popular Indian cheese with a firm texture that holds up well in curries. Here's how to prepare the paneer as a substitute in this recipe:\n\nIngredients:\n- 1 lb (450g) paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tbsp vegetable oil\n- 2 tbsp butter\n- 1 onion, finely chopped\n- 3 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp turmeric\n- 1 tsp paprika\n- 1 tsp garam masala\n- 1 can (14 oz) of tomato sauce\n- 1/2 cup heavy cream\n- Salt and pepper, to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Instead of marinating chicken, marinate the paneer pieces in the yogurt, turmeric, paprika, and a pinch of salt as per the original recipe. Refrigerate for at least 30 minutes.\n2. Follow steps 2-6 of the original recipe, replacing the chicken with th